In [1]:
import pandas as pd
import numpy as np
X = pd.read_csv('../artefacts_copy/BO_initial_X', header=None)
X.columns = ['feature_' + str(i) for i in range(X.shape[1])]
X = X.iloc[:, :-1]

# X = X.iloc[:, :6]

y = pd.read_csv('../artefacts_copy/BO_initial_Y', header=None)
y.columns = ['target']

In [2]:
X.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316
0,115.0,13.0,368.0,4.0,114.0,13.0,588.0,3.0,7.0,1.0,...,16.0,54.0,19.0,8833.0,71.0,14.0,2319.0,2807.0,4.0,68.0
1,122.0,17.0,430.0,4.0,88.0,14.0,548.0,2.0,7.0,5.0,...,22.0,52.0,19.0,7904.0,73.0,21.0,2033.0,3053.0,4.0,44.0
2,71.0,15.0,490.0,4.0,109.0,12.0,564.0,3.0,12.0,2.0,...,14.0,44.0,31.0,6002.0,74.0,30.0,2139.0,2346.0,1.0,66.0
3,103.0,18.0,372.0,5.0,65.0,18.0,572.0,3.0,10.0,3.0,...,18.0,83.0,17.0,5362.0,45.0,21.0,2219.0,2035.0,1.0,53.0
4,73.0,13.0,510.0,5.0,73.0,13.0,514.0,4.0,9.0,5.0,...,29.0,54.0,15.0,7964.0,62.0,24.0,2977.0,2032.0,4.0,55.0


In [3]:
# extracted experimental design

X_cut = X.iloc[:len(X.columns), :]
y_cut = y.iloc[:len(X.columns), :]

X_cut.shape

(317, 317)

In [4]:
X_cut.to_numpy()[:3]

array([[1.15000000e+02, 1.30000000e+01, 3.68000000e+02, 4.00000000e+00,
        1.14000000e+02, 1.30000000e+01, 5.88000000e+02, 3.00000000e+00,
        7.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.20000000e+01,
        4.71000000e+02, 3.55000000e+02, 2.99000000e+02, 5.00000000e+00,
        2.72000000e+02, 7.57000000e+02, 3.56200000e+03, 8.30000000e+01,
        8.30000000e+01, 1.06100000e+03, 7.64000000e+02, 4.00000000e+00,
        1.30000000e+01, 4.30000000e+01, 8.90700000e+04, 1.70000000e+01,
        2.80000000e+02, 7.00000000e+00, 2.00000000e+00, 5.00000000e+00,
        4.56500000e+03, 2.86700000e+03, 9.33000000e+02, 4.44968553e-01,
        0.00000000e+00, 3.60000000e+01, 2.20000000e+01, 0.00000000e+00,
        2.00000000e+00, 3.00000000e+00, 1.00000000e+00, 1.60000000e+01,
        4.00000000e+00, 2.00000000e+00, 1.40000000e+01, 1.80000000e+01,
        1.04000000e+02, 1.50000000e+01, 1.10000000e+01, 3.00000000e+00,
        1.70000000e+01, 3.00000000e+00, 3.00000000e+00, 1.500000

In [5]:
from domain import read_configs, controllable_params, transform_domain
from globals import set_model_type

set_model_type('BO')

read_configs([
        "../networks/configtx/configtx.yaml",
        "../networks/compose/docker/peercfg-org1/core.yaml",
        "../networks/compose/docker/peercfg-org2/core.yaml",
        "../networks/compose/docker/peercfg-org3/core.yaml",
        "../networks/compose/docker/peercfg-org4/core.yaml",
        # "../networks/compose/docker/ordcfg/orderer.yaml",
])

domain = controllable_params#[:6]

Setting model type to: BO


In [6]:
import blackboxoptim as bbo

In [7]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [8]:
regressor = GaussianProcessRegressor(random_state=13)

In [9]:
regressor.fit(X, y)

GaussianProcessRegressor(random_state=13)

In [10]:
len(domain)

317

In [11]:
# from SALib.sample import saltelli
# from SALib.analyze import sobol
# problem = {
#         'num_vars': 15,
#         'names': X.columns.tolist()[:15],
#         'bounds': [item['bounds'] for item in domain][:15],
#     }

# cut_regressor = GaussianProcessRegressor(random_state=13)
# cut_regressor.fit(X, y)

# samples = saltelli.sample(problem, 1024, calc_second_order=False)
# predictions = regressor.predict(samples)
# Si = sobol.analyze(problem, predictions, parallel=True, n_processors=6, calc_second_order=False)
# # print(Si.keys())
# sobol_df = pd.DataFrame({
#         'Feature': problem['names'],
#         'ST': Si['ST']

#     }).sort_values(by='ST', ascending=False)
# sobol_df.reset_index(inplace=True)

In [12]:
# np.min(predictions), np.max(predictions), np.mean(predictions), np.std(predictions), len(predictions)

In [13]:
from pySOT.strategy import DYCORSStrategy
from pySOT.optimization_problems import OptimizationProblem

class CustomProblem(OptimizationProblem): 
            def __init__(self, dim):
                self.dim = dim
                self.lb = np.array([item['bounds'][0] for item in domain])
                self.ub = np.array([item['bounds'][1] for item in domain])
                self.cont_var = np.array([i for i, item in enumerate(domain) if item['type'] == 'continuous'])
                self.int_var = np.array([i for i, item in enumerate(domain) if item['type'] == 'discrete'])
                self.info = "Hyperledger Fabric Tuning"

                print("CustomProblem initialized with dim:", dim)
                print("Lower bounds:", len(self.lb))
                print("Upper bounds:", len(self.ub))

            def eval(self, x):
                print("Objective function called with some x")
                return np.random.uniform(400, 500)
                # raise RuntimeError("This function should not be called directly.")
                return 0.0

In [14]:
from pySOT.experimental_design import LatinHypercube, TwoFactorial

In [15]:
from pySOT.surrogate import GPRegressor

lb = np.array([item['bounds'][0] for item in domain])
ub = np.array([item['bounds'][-1] for item in domain])
regressor_dycors = GPRegressor(dim=len(domain), lb=lb, ub=ub, gp=regressor)
# regressor_dycors.add_points(X.to_numpy(), y.to_numpy())

In [16]:
len(domain), len(lb), len(ub)

(317, 317, 317)

In [29]:
from pySOT.experimental_design.experimental_design import ExperimentalDesign

class EmptyDesign(ExperimentalDesign):
    def __init__(self, dim):
        # super().__init__(dim)
        self.dim = dim
        self.num_pts = dim
        print("Empty design called init")

    def generate_points(self, lb, ub, int_var=None):
        # Always return an empty (0 × dim) array
        # return np.empty((0, self.dim))
        # return np.array([(lb + ub) / 2])
        # return X.to_numpy()
        # print("Empty design called generate with int_var", int_var)
        # latin = LatinHypercube(self.dim, self.dim)
        return X.to_numpy()
        # points1 = np.array([(lb + ub) / 2])
        # points2 = np.array([lb])
        # points = np.vstack((points1, points2))
        # print(points.shape)
        # # points[int_var] = np.round(points[int_var])
        # for i in range(len(points)):
        #     if i in int_var:
        #         points[i] = np.round(points[i])
        #     else:
        #         points[i] = points[i]
        # return points
    
    
    
design = EmptyDesign(dim=len(domain))

Empty design called init


In [30]:
# from pySOT.auxiliary_problems import candidate_dycors

# candidate_dycors(num_pts=1,
#                  opt_prob=CustomProblem(dim=len(domain)),
#                  surrogate=regressor_dycors,
#                  X=X,
#                  fX=y,
#                  )

In [31]:
cube = LatinHypercube(dim=len(domain), num_pts=len(domain))
# cube.generate_points(lb, ub, int_var=None).shape

In [32]:
from pySOT.surrogate import RBFInterpolant

# model = GPRegressor(dim=len(domain), lb=lb, ub=ub)
# model.add_points(X.to_numpy(), y.to_numpy())
# model.predict(X[0, :])

In [33]:
from poap.controller import SerialController
problem = CustomProblem(dim=len(domain))
strat = DYCORSStrategy(max_evals=len(domain) + 300,
                       surrogate=regressor_dycors,
                       opt_prob=problem,
                       exp_design=design,
                       asynchronous=False,
                       batch_size=1,
                       num_cand=1,
                       )
opt = SerialController(objective=problem.eval)
opt.strategy = strat

CustomProblem initialized with dim: 317
Lower bounds: 317
Upper bounds: 317


In [34]:
opt.run()

Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective function called with some x
Objective fu

In [ ]:
# opt = SerialController(objective=problem.eval)
# opt.strategy = strategy

In [ ]:
opt.run()

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

transformer = GaussianRandomProjection(n_components=317, random_state=13)

In [ ]:
X.head(5)

In [ ]:
low_dimentional = transformer.fit_transform(X.head(5))


In [ ]:
recovered_X = transformer.inverse_transform(low_dimentional)
pd.DataFrame(recovered_X, columns=X.columns).head(5)

In [ ]:
from sliced import SlicedInverseRegression

In [ ]:
from sklearn.exceptions import DataConversionWarning
np.int = np.int32

sir = SlicedInverseRegression(n_directions=150)
X_new = sir.fit_transform(X.to_numpy(), y.to_numpy().ravel())

In [ ]:
W = sir.directions_
W

In [ ]:
X_est = X_new @ W

X_est = pd.DataFrame(X_est, columns=['feature_' + str(i) for i in range(X_est.shape[1])])
X_est.head(5)

In [ ]:
# REMBO
projection = np.random.normal(size=(len(domain), 15))

# np.clip()

def custom_objective(X_small):
    recovered_point = projection @ X_small
    recovered_point = np.clip(recovered_point, lb, ub)
    # Bench(recovered_point)

#TODO: use np.clip


In [46]:
domain[:3]

[{'name': 'Orderer.BatchSize.AbsoluteMaxBytes|MB',
  'type': 'discrete',
  'bounds': (63, 122),
  'config idx': 0,
  'default value': 99},
 {'name': 'Orderer.BatchSize.MaxMessageCount',
  'type': 'discrete',
  'bounds': (7, 18),
  'config idx': 0,
  'default value': 10},
 {'name': 'Orderer.BatchSize.PreferredMaxBytes|KB',
  'type': 'discrete',
  'bounds': (357, 668),
  'config idx': 0,
  'default value': 512}]

In [45]:
len(transform_domain(domain))

317

In [53]:
# Entropy Search Acquisition function 'ES' - F this

# import GPyOpt
# from GPyOpt.acquisitions.ES import AcquisitionEntropySearch
# from GPyOpt.util.mcmc_sampler import AffineInvariantEnsembleSampler

# def objective(X):
#     print("Objective function called with some x")
#     return 0.0

# sampler = AffineInvariantEnsembleSampler(domain)

# ES_acquisition = AcquisitionEntropySearch(
#     model=GPyOpt.models.GPModel(),
#     space=GPyOpt.core.space.Design_space(domain=domain),
#     sampler=sampler,
# )

# opt = GPyOpt.methods.BayesianOptimization(
#             f=objective,
#             domain=transform_domain(domain),
#             acquisition_type=ES_acquisition,
#             initial_design_type='random', # placeholder
#             initial_design_numdata=2,
#             maximize=True,
#             num_cores=6,
#             # X=X,
#             # Y=y,
#         )


In [55]:
# opt.run_optimization(max_iter=10)